In [18]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img


# loading facial expression dataset from Kaggle

In [19]:
testData = 'datasetPictures/test/test'
trainData = 'datasetPictures/train/train'
# def loadDataset(directory): 
#     imagePaths = []
#     labels = []

#     for label in os.listdir(directory):
#         for filename in os.listdir(directory+label): 
#             imagePath = os.path.join(directory, label, filename)
#             imagePaths.append(imagePath) #input: image
#             labels.append(label) #output: label

#         print(label, "Completed")
#     return imagePaths, labels

imagesTrain = []
labelsTrain = []
print("Loading Train Dataset")
for label in os.listdir(trainData):
    if not label.startswith('.DS'):
        for picture in os.listdir(trainData+'/'+label):
            picPath = os.path.join(trainData, label, picture)
            imagesTrain.append(picPath) #input: image
            labelsTrain.append(label) #output: label
    print(label, "Completed")

imagesTest = []
labelsTest = []
print("Loading Test Dataset")

for label in os.listdir(testData):
    if not label.startswith('.DS'):
        for picture in os.listdir(testData+'/'+label):
            picPath = os.path.join(testData, label, picture)
            imagesTest.append(picPath) #input: image
            labelsTest.append(label) #output: label
    print(label, "Completed")



Loading Train Dataset
happy Completed
.DS_Store Completed
sad Completed
fear Completed
surprise Completed
neutral Completed
angry Completed
disgust Completed
Loading Test Dataset
happy Completed
.DS_Store Completed
sad Completed
fear Completed
surprise Completed
neutral Completed
angry Completed
disgust Completed


# creating dataframe

In [20]:
# training
train = pd.DataFrame()
train ['image'] = imagesTrain
train['label'] = labelsTrain
# shuffle the dataset
train = train.sample(frac = 1).reset_index(drop = True) 
train.head()

,image,label
0,datasetPictures/train/train/surprise/Training_...,surprise
1,datasetPictures/train/train/happy/Training_170...,happy
2,datasetPictures/train/train/happy/Training_632...,happy
3,datasetPictures/train/train/sad/Training_76072...,sad
4,datasetPictures/train/train/neutral/Training_4...,neutral


In [21]:
#testing
test = pd.DataFrame()
test ['image'] = imagesTest
test['label'] = labelsTest
test.head()

,image,label
0,datasetPictures/test/test/happy/PrivateTest_45...,happy
1,datasetPictures/test/test/happy/PrivateTest_61...,happy
2,datasetPictures/test/test/happy/PublicTest_536...,happy
3,datasetPictures/test/test/happy/PrivateTest_52...,happy
4,datasetPictures/test/test/happy/PublicTest_411...,happy


# Extracting Features

In [22]:
def extractFeatures(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    sampleSize = len(features)
    width = 48
    height = 48
    dimension = 1
    features = features.reshape(sampleSize, width, height, dimension)
    return features

trainFeatures = extractFeatures(train['image'])

  0%|          | 0/28709 [00:00<?, ?it/s]

In [23]:
testFeatures = extractFeatures(test['image'])

  0%|          | 0/7178 [00:00<?, ?it/s]

# Preprocessing data to convert labels (string) to integer values for CNN model

In [29]:
# normalize image so pixels can be in range of 0 and 1
xTrain = trainFeatures/255.0
xTest = testFeatures/255.0

# convert label to integer 
from  sklearn.preprocessing import LabelEncoder 
enc = LabelEncoder()
# stores corresponding mapping of label to an integer
enc.fit(train['label'])
# converts label (string) into an integer 
yTrain = enc.transform(train['label'])
yTest = enc.transform(test['label'])

yTrain = to_categorical(yTrain, num_classes = 7)
yTest = to_categorical(yTest, num_classes = 7)

# configuration 
width = 48
height = 48
dimension = 1
inputShape = (width, height, dimension)
outputClass = 7



In [30]:
yTrain[0]

array([0., 0., 0., 0., 0., 0., 1.], dtype=float32)